In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import numpy as np
import torch
import glob
import os

from bs4 import BeautifulSoup
# data_path = '/Users/ruchiraravishankar/Downloads/iam_dataset/'
data_path = '/content/drive/MyDrive/iam_dataset/'
save_path = "/content/drive/MyDrive/iam_dataset/saved_model"
# print(glob.glob(data_path))

In [ ]:
labelled_data = np.load(data_path + 'labelled_data.npy')

print(labelled_data.shape)

(4901, 2)


In [ ]:
labelled_data_0 = labelled_data[labelled_data[:,1] == '000']

print(labelled_data_0.shape)

labelled_data_0 = labelled_data_0[:196]
print(labelled_data_0.shape)

print(np.unique(labelled_data_0[:,1]))

(693, 2)
(196, 2)
['000']


In [ ]:
remove_000 = labelled_data[labelled_data[:,1]!='000']

print(np.unique(remove_000[:,1]))
print(np.unique(remove_000[:,1]).shape)

['085' '118' '150' '151' '152' '153' '154' '155' '202' '203' '204' '205'
 '206' '207' '208' '209' '247' '248' '273' '274' '315' '332' '333' '334'
 '335' '336' '337' '338' '339' '340' '341' '342' '343' '344' '345' '346'
 '347' '348' '349' '384' '415' '551' '552' '567' '588' '634' '635' '670'
 '671']
(49,)


In [ ]:
y = [remove_000[remove_000[:,1]==k] for k in np.unique(remove_000[:,1])]
print(len(y))

49


In [ ]:
labelled_data_1 = []
for i in y:
    labelled_data_1.append(i[:4])
print(len(labelled_data_1))

labelled_data_1 = np.array(labelled_data_1)

print(labelled_data_1.shape)

49
(49, 4, 2)


In [ ]:
labelled_data_1=labelled_data_1.reshape(-1,2)

In [ ]:
print(labelled_data_1.shape)

(196, 2)


In [ ]:
print(np.unique(labelled_data_1[:,1]))

['085' '118' '150' '151' '152' '153' '154' '155' '202' '203' '204' '205'
 '206' '207' '208' '209' '247' '248' '273' '274' '315' '332' '333' '334'
 '335' '336' '337' '338' '339' '340' '341' '342' '343' '344' '345' '346'
 '347' '348' '349' '384' '415' '551' '552' '567' '588' '634' '635' '670'
 '671']


In [ ]:
labelled_data_new = np.concatenate((labelled_data_0, labelled_data_1))
np.random.shuffle(labelled_data_new)
print(labelled_data_new.shape) # mix of sentences from writer id 0 and others, labelled

(392, 2)


In [ ]:
# split before augmenting to ensure the sentences are being split and not the augmented samples
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(labelled_data_new[:,0], labelled_data_new[:,1], test_size=0.33, random_state=42)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.33, random_state=42)
print(X_train.shape, X_test.shape, X_val.shape, y_train.shape, y_test.shape, y_val.shape)

(175,) (130,) (87,) (175,) (130,) (87,)


In [ ]:
from sklearn.utils import shuffle
from PIL import Image
import random

CROP_SIZE = 113
NUM_LABELS = 50

def get_augmented_sample(sample, label, sample_ratio):
    # Get current image details
    img = Image.open(sample)
    img_width = img.size[0]
    img_height = img.size[1]

    # Compute resize dimensions such that aspect ratio is maintained
    height_fac = CROP_SIZE / img_height
    size = (int(img_width * height_fac), CROP_SIZE)

    # Resize image 
    new_img = img.resize((size), Image.ANTIALIAS)
    new_img_width = new_img.size[0]
    new_img_height = new_img.size[1]

    # Generate a random number of crops of size 113x113 from the resized image
    x_coord = list(range(0, new_img_width - CROP_SIZE))
    num_crops = int(len(x_coord) * sample_ratio)
    random_x_coord = random.sample(x_coord, num_crops)
    
    # Create augmented images (cropped forms) and map them to a label (writer)
    images = []
    labels = []
    for x in random_x_coord:
        img_crop = new_img.crop((x, 0, x + CROP_SIZE, CROP_SIZE))
        # Transform image to an array of numbers
        images.append(np.asarray(img_crop))
        labels.append(label)

    return images, labels

In [ ]:
def augment(data, labels):
    augmented_sample_list = []
    augmented_label_list = []
    for i in range(len(data)):
        augmented_samples, augmented_labels = get_augmented_sample(data[i], labels[i],0.1)
        augmented_sample_list.append(augmented_samples)
        augmented_label_list.append(augmented_labels)
    return augmented_sample_list, augmented_label_list

In [ ]:
augsamps, auglbls = augment(X_train, y_train) # returns two lists of arrays
augsamps_test, auglbls_test = augment(X_test, y_test)
augsamps_val, auglbls_val = augment(X_val, y_val)

In [ ]:
import operator
from functools import reduce

# flattening the lists
aug_samps = reduce(operator.add, augsamps)
aug_lbls = reduce(operator.add, auglbls)

aug_samps_test = reduce(operator.add, augsamps_test)
aug_lbls_test = reduce(operator.add, auglbls_test)

aug_samps_val = reduce(operator.add, augsamps_val)
aug_lbls_val = reduce(operator.add, auglbls_val)

print(len(aug_samps))
print(len(aug_lbls))

print(len(aug_samps_test))
print(len(aug_lbls_test))

print(len(aug_samps_val))
print(len(aug_lbls_val))

30378
30378
23373
23373
14671
14671


In [ ]:
# converting lists to arrays
aug_samps_arr = np.array(aug_samps)
aug_lbls_arr = np.array(aug_lbls)
aug_samps_arr_test = np.array(aug_samps_test)
aug_lbls_arr_test = np.array(aug_lbls_test)
aug_samps_arr_val = np.array(aug_samps_val)
aug_lbls_arr_val = np.array(aug_lbls_val)

print(aug_samps_arr.shape)
print(aug_lbls_arr.shape)
print(aug_samps_arr_test.shape)
print(aug_lbls_arr_test.shape)
print(aug_samps_arr_val.shape)
print(aug_lbls_arr_val.shape)

(30378, 113, 113)
(30378,)
(23373, 113, 113)
(23373,)
(14671, 113, 113)
(14671,)


In [ ]:
X_train = aug_samps_arr
X_train = X_train.reshape(X_train.shape[0], 1, CROP_SIZE, CROP_SIZE)
X_train = X_train.astype('float16')
X_train /= 255.0
y_train = aug_lbls_arr


X_test = aug_samps_arr_test
X_test = X_test.reshape(X_test.shape[0], 1, CROP_SIZE, CROP_SIZE)
X_test = X_test.astype('float16')
X_test /= 255.0
y_test = aug_lbls_arr_test

X_val = aug_samps_arr_val
X_val = X_val.reshape(X_val.shape[0], 1, CROP_SIZE, CROP_SIZE)
X_val = X_val.astype('float16')
X_val /= 255.0
y_val = aug_lbls_arr_val



In [ ]:
print(X_train.shape)
print(y_train.shape)
print(X_test.shape)
print(y_test.shape)
print(X_val.shape)
print(y_val.shape)

(30378, 1, 113, 113)
(30378,)
(23373, 1, 113, 113)
(23373,)
(14671, 1, 113, 113)
(14671,)


In [ ]:
new = np.unique(y_train)
print(new)

y_train[y_train !='000'] = 1
new = np.unique(y_train)
print(new)

y_train[y_train=='000'] = 0
new = np.unique(y_train)
print(new)

print(y_train.shape)

y_train = y_train.astype('float16')

print(np.unique(y_train))

['000' '085' '118' '150' '151' '152' '154' '155' '202' '203' '204' '206'
 '207' '208' '209' '247' '273' '274' '332' '333' '334' '335' '336' '338'
 '339' '340' '341' '342' '343' '344' '345' '346' '347' '348' '349' '384'
 '415' '551' '552' '567' '588' '634' '635' '670' '671']
['000' '1']
['0' '1']
(30378,)
[0. 1.]


In [ ]:
new = np.unique(y_test)
print(new)

y_test[y_test !='000'] = 1
new = np.unique(y_test)
print(new)

y_test[y_test=='000'] = 0
new = np.unique(y_test)
print(new)

print(y_test.shape)

y_test = y_test.astype('float16')

print(np.unique(y_test))

['000' '085' '118' '150' '151' '153' '154' '155' '202' '203' '204' '205'
 '207' '209' '247' '248' '273' '274' '315' '333' '334' '335' '337' '338'
 '340' '341' '343' '344' '345' '346' '347' '348' '349' '384' '415' '551'
 '552' '567' '588' '634' '635' '670' '671']
['000' '1']
['0' '1']
(23373,)
[0. 1.]


In [ ]:
new = np.unique(y_val)
print(new)

y_val[y_val !='000'] = 1
new = np.unique(y_val)
print(new)

y_val[y_val=='000'] = 0
new = np.unique(y_val)
print(new)

print(y_val.shape)

y_val = y_val.astype('float16')

print(np.unique(y_val))

['000' '085' '151' '152' '153' '155' '202' '203' '204' '205' '206' '207'
 '209' '247' '248' '273' '332' '333' '336' '337' '338' '339' '340' '342'
 '343' '345' '346' '348' '349' '415' '552' '588' '634' '671']
['000' '1']
['0' '1']
(14671,)
[0. 1.]


In [ ]:
torch.manual_seed(42)
CUDA = True
device = torch.device('cuda' if CUDA else 'cpu')
kwargs = {'num_workers': 1, 'pin_memory': True} if CUDA else {}
device

device(type='cuda')

In [ ]:
from torch.utils.data import TensorDataset, DataLoader


tensor_x_train = torch.Tensor(X_train).type(torch.cuda.FloatTensor) # transform to torch tensor
tensor_y_train = torch.Tensor(y_train).type(torch.cuda.FloatTensor).long()



# tensor_x_train = torch.Tensor(X_train) # transform to torch tensor
# tensor_y_train = torch.Tensor(y_train).long()

train_dataset = TensorDataset(tensor_x_train,tensor_y_train) # create your datset
train_dataloader = DataLoader(train_dataset,batch_size=16, shuffle=True, drop_last=True) # create your dataloader

In [ ]:
tensor_x_val = torch.Tensor(X_val).type(torch.cuda.FloatTensor) # transform to torch tensor
tensor_y_val = torch.Tensor(y_val).type(torch.cuda.FloatTensor).long()
# tensor_x_train = torch.Tensor(X_train) # transform to torch tensor
# tensor_y_train = torch.Tensor(y_train).long()

val_dataset = TensorDataset(tensor_x_val,tensor_y_val) # create your datset
val_dataloader = DataLoader(val_dataset,batch_size=16, shuffle=True, drop_last=True) # create your dataloader

In [ ]:
tensor_x_test = torch.Tensor(X_test).type(torch.cuda.FloatTensor) # transform to torch tensor
tensor_y_test = torch.Tensor(y_test).long().type(torch.cuda.FloatTensor).long()

# tensor_x_test = torch.Tensor(X_test) # transform to torch tensor
# tensor_y_test = torch.Tensor(y_test).long()

test_dataset = TensorDataset(tensor_x_test,tensor_y_test) # create your datset
test_dataloader = DataLoader(test_dataset, batch_size=16, shuffle=False, drop_last=True) # create your dataloader

In [ ]:
tensor_x_train.shape

torch.Size([30378, 1, 113, 113])

In [ ]:
import torch
from torch import nn
class Binary_Classifier(nn.Module): 
    def __init__(self):
        super(Binary_Classifier, self).__init__()
        self.conv1 = nn.Conv2d(in_channels=1, out_channels=10, kernel_size=3)
        self.conv2 = nn.Conv2d(10, 20, kernel_size=3)
        self.conv_drop = nn.Dropout2d()
        self.conv3 = nn.Conv2d(20, 30, kernel_size=3)
        self.fc1 = nn.Linear(4320, 1024)
        self.fc2 = nn.Linear(1024, 2)
        self.relu = nn.ReLU()
        self.max_pool2d = nn.MaxPool2d(2)
    def forward(self, x):
        x = self.relu(self.max_pool2d(self.conv1(x)))
        x = self.relu(self.max_pool2d(self.conv_drop(self.conv2(x))))
        x = self.relu(self.max_pool2d(self.conv_drop(self.conv3(x))))
        x = x.view(x.shape[0],-1)
        # print(x.shape)
        x = self.relu(self.fc1(x))
        x = self.conv_drop(x)
        x = self.fc2(x)
        return x

In [ ]:
model=Binary_Classifier().to(device)
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(),lr = 0.0001)

# import torch.optim.lr_scheduler.StepLR
# scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=1, gamma=0.1)


In [ ]:
import os

if os.path.exists(save_path +'/model_3_convs.pt'):
    print('Loading checkpoint: %s' %save_path +'/model_3_convs.pt')
    path = torch.load(save_path +'/model_3_convs.pt')
    epoch = path['epoch']
    model.load_state_dict(path['model'])
    optimizer.load_state_dict(path['optimizer'])
else:
  print("DID NOT DO ANYTHING")

Loading checkpoint: /content/drive/MyDrive/iam_dataset/saved_model/model_3_convs.pt


In [ ]:
result = []
prediction = []
model.eval()
with torch.no_grad():
    for idx, (data, target) in enumerate(test_dataloader):
        output = model(data)
        output_sm = nn.Softmax(dim=1)(output)
        _, predicted = torch.max(output, 1)
        prediction.append(predicted)
        result.append(output_sm)
        
result = torch.stack(result).detach().cpu().numpy()
prediction = torch.stack(prediction).detach().cpu().numpy()

print(result.shape, prediction.shape)

(1460, 16, 2) (1460, 16)


In [ ]:
!pip install captum
from captum.attr import GuidedGradCam
from captum.attr import visualization as viz
from captum.attr import IntegratedGradients
from captum.attr import Saliency

     |████████████████████████████████| 1.4 MB 5.2 MB/s 


In [ ]:
def gradcam(model, data, target, map, sign):
  data = data.reshape(1, data.shape[0], data.shape[1], data.shape[2])
  guided_gc = GuidedGradCam(model, model.conv3)
  # inp = data
  # tar = tensor_y_test[0]
  attribution = guided_gc.attribute(data, target)
  # print(attribution)
  # print(attribution.cpu().detach().numpy().reshape(1,113,113))
  _ = viz.visualize_image_attr(attribution.cpu().detach().numpy().reshape(1,113,113), data.cpu().numpy().reshape(1,113,113), map, sign)
  return attribution

In [ ]:
gradcam(model, tensor_x_test[7], tensor_y_test[7],"heat_map", "all")

/usr/local/lib/python3.7/dist-packages/captum/_utils/gradient.py:58: UserWarning: Input Tensor 0 did not already require gradients, required_grads has been set automatically.
  "required_grads has been set automatically." % index
/usr/local/lib/python3.7/dist-packages/captum/attr/_core/guided_backprop_deconvnet.py:60: UserWarning: Setting backward hooks on ReLU activations.The hooks will be removed after the attribution is finished
  "Setting backward hooks on ReLU activations."


tensor([[[[-0., 0., -0.,  ..., 0., 0., 0.],
          [0., -0., 0.,  ..., 0., 0., 0.],
          [-0., 0., 0.,  ..., 0., 0., 0.],
          ...,
          [0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.]]]], device='cuda:0',
       grad_fn=<MulBackward0>)
[[[-0.  0. -0. ...  0.  0.  0.]
  [ 0. -0.  0. ...  0.  0.  0.]
  [-0.  0.  0. ...  0.  0.  0.]
  ...
  [ 0.  0.  0. ...  0.  0.  0.]
  [ 0.  0.  0. ...  0.  0.  0.]
  [ 0.  0.  0. ...  0.  0.  0.]]]


tensor([[[[-0., 0., -0.,  ..., 0., 0., 0.],
          [0., -0., 0.,  ..., 0., 0., 0.],
          [-0., 0., 0.,  ..., 0., 0., 0.],
          ...,
          [0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.]]]], device='cuda:0',
       grad_fn=<MulBackward0>)

In [ ]:
def intgrad(model, data, target):
  trial = data.reshape(1, data.shape[0], data.shape[1], data.shape[2])
  ig = IntegratedGradients(model)
  baseline1 = torch.tensor([0.0]).type(torch.cuda.FloatTensor)
  attribution = ig.attribute(trial, baseline1, target)
  _ = viz.visualize_image_attr(attribution.cpu().numpy().reshape(1,113,113), data.cpu().numpy(),"heat_map")
  return attribution

In [ ]:
intgrad(model, tensor_x_test[0], tensor_y_test[0])

tensor([[[[ 1.8849e-04,  5.6688e-04, -7.4173e-04,  ..., -2.8623e-10,
            0.0000e+00,  0.0000e+00],
          [-5.8122e-04, -2.9903e-05, -8.9487e-04,  ..., -1.2855e-09,
            0.0000e+00,  0.0000e+00],
          [ 3.7448e-04, -9.9413e-04,  2.1899e-04,  ...,  6.6703e-10,
            0.0000e+00,  0.0000e+00],
          ...,
          [-6.5343e-10, -6.8793e-10, -5.8805e-10,  ..., -2.2761e-10,
            0.0000e+00,  0.0000e+00],
          [ 0.0000e+00,  0.0000e+00,  0.0000e+00,  ...,  0.0000e+00,
            0.0000e+00,  0.0000e+00],
          [ 0.0000e+00,  0.0000e+00,  0.0000e+00,  ...,  0.0000e+00,
            0.0000e+00,  0.0000e+00]]]], device='cuda:0', dtype=torch.float64)

In [ ]:
def saliency(model, data, target):
  data = data.reshape(1, data.shape[0], data.shape[1], data.shape[2])
  sal = Saliency(model)
# baseline1 = torch.tensor([0.0]).type(torch.cuda.FloatTensor)
  attribution = sal.attribute(data, target)
  _ = viz.visualize_image_attr(attribution.cpu().numpy().reshape(1,113,113), data.cpu().numpy().reshape(1,113,113),"blended_heat_map")
  return attribution

In [ ]:
saliency(model, tensor_x_test[0], tensor_y_test[0])

/usr/local/lib/python3.7/dist-packages/captum/_utils/gradient.py:58: UserWarning: Input Tensor 0 did not already require gradients, required_grads has been set automatically.
  "required_grads has been set automatically." % index


tensor([[[[8.6294e-04, 3.2247e-03, 2.4986e-03,  ..., 0.0000e+00,
           0.0000e+00, 0.0000e+00],
          [2.9717e-03, 8.5392e-04, 4.5632e-03,  ..., 0.0000e+00,
           0.0000e+00, 0.0000e+00],
          [2.1495e-03, 6.2844e-03, 1.9779e-05,  ..., 0.0000e+00,
           0.0000e+00, 0.0000e+00],
          ...,
          [0.0000e+00, 0.0000e+00, 0.0000e+00,  ..., 0.0000e+00,
           0.0000e+00, 0.0000e+00],
          [0.0000e+00, 0.0000e+00, 0.0000e+00,  ..., 0.0000e+00,
           0.0000e+00, 0.0000e+00],
          [0.0000e+00, 0.0000e+00, 0.0000e+00,  ..., 0.0000e+00,
           0.0000e+00, 0.0000e+00]]]], device='cuda:0')

In [ ]:
for idx, (data, target) in enumerate(zip(tensor_x_test, tensor_y_test)):
  saliency(model, data, target)

/usr/local/lib/python3.7/dist-packages/captum/_utils/gradient.py:58: UserWarning: Input Tensor 0 did not already require gradients, required_grads has been set automatically.
  "required_grads has been set automatically." % index


KeyboardInterrupt: ignored